<a href="https://colab.research.google.com/github/VallabhK/ExperimentsWithData/blob/master/FutureSalesPrediction1CCompany.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Predict Future Sales**

---

**Problem Statement:** In this competition you will work with a challenging time-series dataset consisting of daily sales data, kindly provided by one of the largest Russian software firms - 1C Company. 

We are asking you to predict total sales for every product and store in the next month. Make sure the model is robust enough to handle slight changes in list of shops and products in the shop



In [0]:
#Import required modules
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [0]:
url = 'https://raw.githubusercontent.com/VallabhK/ExperimentsWithData/master/Input/item_categories.csv'
df1 = pd.read_csv(url);

In [0]:
https://databricks.com/product/faq/community-edition
